In [ ]:
from src.utils.gee import init_gee

init_gee()

In [ ]:
import datetime as dt
import ee
import geemap
from src.data.gee.collections import get_s1_collection
from typing import Union
from shapely.geometry import Polygon
from pathlib import Path
from src.utils.gee import shapely_to_gee, custom_mosaic_same_day

def get_s1_collection(geometry_ee: ee.Geometry, start_date: Union[dt.date, str], end_date: Union[dt.date, str]):

    start_date = start_date.strftime("%Y-%m-%d") if isinstance(start_date, dt.date) else start_date
    end_date = end_date.strftime("%Y-%m-%d") if isinstance(end_date, dt.date) else end_date

    s1 = ee.ImageCollection("COPERNICUS/S1_GRD")  # decibel, orthorectified
    s1_target = (
        s1.filter(ee.Filter.eq("instrumentMode", "IW"))
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
        .filter(ee.Filter.eq("platform_number", "A"))
        .filterDate(start_date, end_date)
        .filterBounds(geometry_ee)
        .map(lambda img: img.select(["VV", "VH"]))
    )

    # add metadata as attributes
    s1_target = (
        s1_target.map(lambda img: img.set("id_", img.id()))
        .map(lambda img: img.set("date", ee.Date(img.date()).format("YYYY-MM-dd")))
        .map(lambda img: img.set("orbit_number", ee.Number(img.get("relativeOrbitNumber_start"))))
        .map(lambda img: img.set("orbit_direction", ee.String(img.get("orbitProperties_pass"))))
    )
    return s1_target

def custom_download_ee_image_collection(
    collection: ee.ImageCollection, out_dir: Path, region: ee.Geometry, scale: int = None, crs=None
):
    """Same as geemap.download_ee_image_collection, but with a try/except to avoid crashing"""

    if not isinstance(collection, ee.ImageCollection):
        raise ValueError("ee_object must be an ee.ImageCollection.")

    if out_dir is None:
        out_dir = Path(".")
    out_dir.mkdir(parents=True, exist_ok=True)

    count = int(collection.size().getInfo())
    print(f"Total number of images: {count}\n")

    col_list = collection.toList(count)
    list_files_error = []
    for i in range(count):
        image = ee.Image(col_list.get(i))
        name = image.get("system:index").getInfo() + ".tif"
        filename = out_dir / name
        print(f"Downloading {i + 1}/{count}: {name}")
        try:
            geemap.download_ee_image(image, filename, region=region, scale=scale, crs=crs)
        except Exception as e:
            print(f"Error downloading {name}: {e}")
            list_files_error.append(name)
    return list_files_error

def download_s1(
    geometry: Union[Polygon, ee.Geometry],
    date: Union[dt.date, str],
    folder: Union[str, Path],
):
    """
    Download Sentinel-1 tile for a given date and geometry.

    Args:
        geometry (Union[Polygon, ee.Geometry]): The geometry. (in EPSG:4326)
        start_date (Union[dt.date, str]): The start date.
        folder (Union[str, Path]): Path to the folder.
    """

    # Check input
    geometry_ee = geometry if isinstance(geometry, ee.Geometry) else shapely_to_gee(geometry)
    folder = Path(folder) if not isinstance(folder, Path) else folder

    start_date = dt.date.fromisoformat(date) if isinstance(date, str) else date
    end_date = start_date + dt.timedelta(days=1)
    s1 = get_s1_collection(geometry_ee, start_date, end_date)

    n_tiles = s1.size().getInfo()
    if n_tiles == 0:
        print("No tiles found.")
        return
    elif n_tiles > 1:
        print(f"Warning: More than one tile found: {n_tiles}")
        s1 = custom_mosaic_same_day(s1)

    return custom_download_ee_image_collection(s1, out_dir=folder, region=geometry_ee, scale=10, crs=None)

In [ ]:
from src.data.unosat import get_unosat_geometry
geo = get_unosat_geometry('UKR1')
date = '2022-03-24'
folder = Path('./quick_data')
folder.mkdir(exist_ok=True)
download_s1(geo, date, folder)

In [ ]:
def download_s2(
    geometry: Union[Polygon, ee.Geometry],
    date: Union[dt.date, str],
    folder: Union[str, Path],
):
    """
    Download Sentinel-2 tilesimage for a given date and geometry.

    Args:
        geometry (Union[Polygon, ee.Geometry]): The geometry. (in EPSG:4326)
        start_date (Union[dt.date, str]): The start date.
        folder (Union[str, Path]): Path to the folder.
    """

    # Check input
    geometry_ee = geometry if isinstance(geometry, ee.Geometry) else shapely_to_gee(geometry)
    folder = Path(folder) if not isinstance(folder, Path) else folder

    start_date = dt.date.fromisoformat(date) if isinstance(date, str) else date
    end_date = start_date + dt.timedelta(days=1)
    s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(geometry_ee).filterDate(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    n_tiles = s2.size().getInfo()
    if n_tiles == 0:
        print("No tiles found.")
        return
    elif n_tiles > 1:
        print(f"Warning: More than one tile found: {n_tiles}")

    return custom_download_ee_image_collection(s2, out_dir=folder, region=geometry_ee, scale=10, crs=None)

In [ ]:
from src.data.unosat import get_unosat_geometry
geo = get_unosat_geometry('UKR1')
date = '2022-01-08'
folder = Path('./quick_data')
folder.mkdir(exist_ok=True)
download_s2(geo, date, folder)